# This jupyter notebook scrapes unstructured data from the website "The Political Graveyard" (www.thepoliticalgraveyard.com) into an excel spreadsheet
## The historical dataset that is generates is about all clergy politicians born in the United States by their birth state and county as well as lived state.

### You will need to download chromedriver to exceute this notebook

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import requests

In [39]:
# Using Selenium for web-scraping
# save the path to chromedriver here
EXE_PATH = r'C:\Users\nikhi\Desktop\Summer 2020\RA Work\Scraping\chromedriver.exe'
browser = webdriver.Chrome(executable_path=EXE_PATH)

# list of all possible clergy positions
clergy_posts=['Pastor', 'Chaplain', 'chaplain', 'pastor','Priest','priest', 'Pastor', 'Minister', 'minister', 'missionary', 'Missionary', 'Rabbi', 'rabbi', 'bishop', 'Bishop', 'Salvation Army officer' , 'cleric', 'Cleric', 'Preacher', 'preacher', 'preacher', 'Clergyman', 'clergyman']

# a dictionary for data entries
clergy_pol = {}

# counter for dictionary entry
count = 0

url = "http://politicalgraveyard.com/occ/clergy.html"
    
# open the link to the webpage
browser.get(url)
browser.implicitly_wait(5)

# read HTML content of the webpage
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

# get all the links in that reigion's webpage
link = soup.findAll('a')

# a list of all the links for various states for that religion 
state_links=[]

# loop over all the links in that religion's webpage
# the strategy is to keep opening the 'geo' links until 'The Political Grveyard' link appears 
# when the loop breaks

for each in link:
    if('The Political Graveyard' in each.text):
        break
    elif(('geo' in each.get('href')) == True):
        # get the link for the states and append to the list
        state_links.append(each.get('href'))

# loop over all the 'geo' links in that religion's webpage
# the strategy is to keep opening the 'geo' links and extract 
# state name and politician's name from the webpage

for st in state_links:
    try:
        html = requests.get(st)
        soup = BeautifulSoup(html.content)

        # identifier for state name
        a = 'Politicians in'

        # loop over all the <\p> tags to identify state's name in each webpage
        for x in (soup.find_all('p')):
                state = str(x)[str(x).find(a)+15:str(x).find('</p>')]

        # open the state link
        browser.get(st)
        browser.implicitly_wait(5)
            
        html = urlopen(st)
        soup = BeautifulSoup(html, 'lxml')

        #  save the html content as a string
        string = str(soup)

        # find all the bold texts in the page
        name = soup.findAll('b')

        # flag to identify whether the bold text is a new name or modified version of the old name
        # Here I use the fact that new names appear only after the religion name is mentioned in the bold text
        flag = 1
            
        # list of all politician names found on the religion by state webpage
        pol_list=[]

        # loop over all the bold text tags
        for each in name:

            # if the text contains the word 'Political', it means that all the names have been scraped
            if('Political' in each.text):
                break

            # if next bold tag is a new name, flag = 1
            if flag==1:
                if (any(x in clergy_posts for x in each.text.split())):
                    continue

                else: 
                    if ('(' in each.text):
                        # if text contains info on birth/death year in (), remove that and save the data
                        pol_list.append(each.text[0:each.text.find('(')])
                    else: 
                        pol_list.append(each.text)

                    # next tag will not be a new name
                    flag = 0

            elif (any(x in clergy_posts for x in each.text.split())):
                # if the bold tag contains a religion name, the next bold tag will have a new name
                flag = 1
        # locate the positions of all the politician names in the text of the html content
        locate=[]
        for each in pol_list:
            locate.append(string.find(each))
        
        l = len(locate)
        for x in np.arange(l):
            if x<=l-2:
                soup = BeautifulSoup(string[locate[x]:locate[x+1]],features="html.parser")
                for each in (soup.find_all('a')):
                    if 'born' in each.get('href'): # if the link is a 'born' link
                        temp1 = string[locate[x]:locate[x+1]].find(each.get_text()) # index of the link in the substring
                        temp2 = string[locate[x]:locate[x+1]].find('/a>',temp1)+5 # index of '/a>' after the link in the substring
                        temp3 = string[locate[x]:locate[x+1]].find('<a',temp2) # index of '<a' after '/a>' in the substring
                        temp4 = string[locate[x]:locate[x+1]][temp2:temp3].replace('\n', ' ') 
                        # extract the born state name located between temp2 and temp3 by replacing the '\n' with ' '

                        dic_pol_name_state_rel[count] = {'lived_state':state, 'born_state':temp4[:temp4.rfind(',')], 'born_county':each.get_text().replace('\n', ' '), 'religion':rel, 'pol_name':pol_list[x] }
                        count +=1

            elif x ==l-1: # if it is the last name in the list of politicians for that state
                # read the substring of 'string' as an html file through BeautifulSoup
                soup = BeautifulSoup(string[locate[x]:],features="html.parser")
                for each in (soup.find_all('a')):  # for each link in the substring
                    if 'born' in each.get('href'): # if the link is a 'born' link
                        temp1 = string[locate[x]:].find(each.get_text()) # index of the link in the substring
                        temp2 = string[locate[x]:].find('/a>',temp1)+5 # index of '/a>' after the link in the substring
                        temp3 = string[locate[x]:].find('<a',temp2) # index of '<a' after '/a>' in the substring
                        temp4 = string[locate[x]:][temp2:temp3].replace('\n', ' ')
                        # extract the born state name located between temp2 and temp3 by replacing the '\n' with ' '

                        dic_pol_name_state_rel[count] = {'lived_state':state, 'born_state':temp4[:temp4.rfind(',')], 'born_county':each.get_text().replace('\n', ' '), 'religion':rel, 'pol_name':pol_list[x] }
                        count +=1

    except NoSuchElementException:
        pass

In [40]:
# save the dictionary into a dataframe
df_clergy_pol=pd.DataFrame.from_dict(clergy_pol).T

# save the dataframe into an excel file
df_clergy_pol.to_excel('clergy_pol.xlsx',engine='xlsxwriter')

In [41]:
# open an excel file that abbreviations for US state names into dataframe
df_st = pd.read_excel('state_name_abb.xlsx')
df_st['ABB'] = df_st['ABB'].str.replace(' ','')
for abb in df_clergy_pol['born_state']:
    # for each abbreviation in the clergy names dataframe
    flag = 0
    for each in df_st['ABB']:
        # for each state in US
        if str(each) in str(abb):
            # if part of the state name is in an actual state name
            # replace the state name (abbreviated) to full state name
            temp = df_st.loc[df_st['ABB']==each].iloc[0,0]
            df_clergy_pol['born_state'] = df_clergy_pol['born_state'].replace(abb, temp)
            flag = 1

In [42]:
# save the dataframe into an excel file
df_clergy_pol.to_excel('clergy_pol_state.xlsx',engine='xlsxwriter')